In [7]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [8]:
from sailor import RouteGenConfig, SailorDataWarehouse

context="ERP admin panel"

_config = RouteGenConfig.from_env()
_warehouse = SailorDataWarehouse(_config, context, db_path="../db")

routes = await _warehouse.create_routes(count=20)
sessions_group = await _warehouse.create_sessions(routes, count=500)

In [9]:
from typing import List
from sklearn.model_selection import train_test_split
from sailor.types.route_specs import SessionSpec

train_sessions: List[SessionSpec] = []
test_sessions: List[SessionSpec] = []

for _sessions in sessions_group:
    if len(_sessions) <= 1:
        train_sessions.extend(_train)
        continue

    _train, _test = train_test_split(_sessions, shuffle=True, test_size=0.2, random_state=14)
    train_sessions.extend(_train)
    test_sessions.extend(_test)

print(f"Train sessions: {len(train_sessions)}")
print(f"Test sessions: {len(test_sessions)}")

Train sessions: 8000
Test sessions: 2000


In [10]:
from sailor.sailor_engine import KNNSailorEngine, SVCSailorEngine

knn_engine = KNNSailorEngine()
knn_engine.fit(routes, train_sessions)

svc_engine = SVCSailorEngine()
svc_engine.fit(routes, train_sessions)

Pipeline(steps=[('tfidf', TfidfVectorizer(stop_words='english')),
                ('svc', LinearSVC(class_weight='balanced', max_iter=2000))])

In [ ]:
from abc import ABC, abstractmethod
from typing import Tuple
import pandas as pd
from sailor.sailor_engine import SailorEngine
from sklearn.metrics import classification_report, top_k_accuracy_score

_top_k_score = 5
class ModelValidator(ABC):
    def __init__(self, model: SailorEngine, sessions: List[SessionSpec]):
        self.model = model
        self._test_queries = [s.context for s in sessions]
        self._targets = model.documentor.transform([s.target for s in sessions])
        self._target_names = [
            model.documentor.get_route(id).path
            for id in model.documentor.labels_
        ]

    @abstractmethod
    def predict(self) -> Tuple: ...

    def test_predict(self, query: str):
        routes = self.model.predict(query)[:_top_k_score]

        print(f"\n=== Test query ===")
        print(f"Query: {query}")
        for route in routes:
            print(f"Path: {route.path}, Score: {route.score:.2f}")
            print(f"Context: {route.context}")

    def validate(self):
        predictions, pred_scores = self.predict()
        route_validations = []

        report = classification_report(
            self._targets, predictions,
            target_names=self._target_names,
            zero_division=0,
            output_dict=True)

        for route_id, route in self.model.documentor._routes.items():
            assert isinstance(report, dict)
            metrics = report.get(route.path, {})

            all_tokens = route.context.split()
            unique_tokens = len(set(all_tokens))
            unique_ratio = unique_tokens / len(all_tokens)

            route_validations.append({
                "path": route.path,
                "tokens": len(all_tokens),
                "unique_tokens": unique_tokens,
                "unique_ratio": round(unique_ratio, 2),
                "f1-score": round(metrics.get("f1-score", 0), 2),
                "precision": round(metrics.get("precision", 0), 2),
                "recall": round(metrics.get("recall", 0), 2),
            })

        df = pd.DataFrame(route_validations)
        df = df.sort_values("f1-score", ascending=False)

        print(f"\n=== {self.model.__class__.__name__} Analysis ===")
        display(df)

        print(f"Average tokens per route: {df['tokens'].mean():.1f}")
        print(f"Average unique tokens per route: {df['unique_tokens'].mean():.1f}")
        print(f"Average unique ratio: {df['unique_ratio'].mean():.2f}")
        print(f"Average F1 score: {df['f1-score'].mean():.2f}")
        print(f"Average precision: {df['precision'].mean():.2f}")
        print(f"Average recall: {df['recall'].mean():.2f}")

        _top_k_accuracy = top_k_accuracy_score(self._targets, pred_scores, k=_top_k_score)
        print(f"Top-{_top_k_score} accuracy: {_top_k_accuracy:.2f}")

        correlation_metrics = [
            ("unique_ratio", "f1-score"),
            ("tokens", "recall"),
            ("unique_tokens", "f1-score")
        ]

        print("\n=== Correlations between tokens and metrics ===")
        for metric1, metric2 in correlation_metrics:
            corr = df[metric1].corr(df[metric2])
            print(f"Correlation {metric1} vs {metric2}: {corr:.3f}")

In [12]:
class _KNNModelValidator(ModelValidator):
    def __init__(self):
        super().__init__(knn_engine, test_sessions)

    def predict(self) -> Tuple:
        predictions = self.model.pipeline.predict(self._test_queries)
        pred_proba = self.model.pipeline.predict_proba(self._test_queries)
        return predictions, pred_proba

_knn_engine = _KNNModelValidator()
_knn_engine.validate()

_knn_engine.test_predict("Create a new user in the system")


=== KNNSailorEngine Analysis ===


,path,tokens,unique_tokens,unique_ratio,f1-score,precision,recall
2,/admin/users/create,728,90,0.12,0.94,0.95,0.93
5,/admin/products/create,708,74,0.10,0.93,0.89,0.98
8,/admin/orders/:id,556,50,0.09,0.92,0.85,0.99
0,/admin/dashboard,567,101,0.18,0.89,0.89,0.89
19,/admin/support,753,212,0.28,0.89,0.92,0.85
12,/admin/reports/sales,707,189,0.27,0.84,0.82,0.87
7,/admin/orders,649,221,0.34,0.83,0.96,0.73
18,/admin/invoices/create,812,160,0.20,0.83,0.76,0.91
10,/admin/customers/:id,685,86,0.13,0.82,0.73,0.95
11,/admin/reports,804,254,0.32,0.81,0.79,0.84


Total routes: 20
Average tokens per route: 706.0
Average unique tokens per route: 169.2
Average unique ratio: 0.24
Average F1 score: 0.80
Average precision: 0.82
Average recall: 0.80
Top-5 accuracy: 0.99

=== Correlations between tokens and metrics ===
Correlation unique_ratio vs precision: -0.028
Correlation tokens vs recall: 0.137
Correlation unique_tokens vs f1-score: -0.585

=== Test query ===
Query: Create a new user in the system
Path: /admin/users/create, Score: 0.31
Context: admin users create users create erp admin new account add user Adding user account Need add new user specific roles New user setup create new user account Creaeting new user add user New user creation Setup new user New user registration New user account creation New user onboarding New user creation Create new employee profile New employee profile User creation form Adding new user account New user setup process New user creation process New employee setup Set new user account finance department Add user d

In [13]:
class _SVCModelValidator(ModelValidator):
    def __init__(self):
        super().__init__(svc_engine, test_sessions)

    def predict(self) -> Tuple:
        predictions = self.model.pipeline.predict(self._test_queries)
        _decisions = self.model.pipeline.decision_function(self._test_queries)
        return predictions, _decisions

_svc_engine = _SVCModelValidator()
_svc_engine.validate()

_svc_engine.test_predict("Create a new user in the system")


=== SVCSailorEngine Analysis ===


,path,tokens,unique_tokens,unique_ratio,f1-score,precision,recall
5,/admin/products/create,708,74,0.10,0.94,0.92,0.97
2,/admin/users/create,728,90,0.12,0.91,0.89,0.93
0,/admin/dashboard,567,101,0.18,0.90,0.88,0.91
19,/admin/support,753,212,0.28,0.88,0.90,0.87
12,/admin/reports/sales,707,189,0.27,0.85,0.81,0.88
8,/admin/orders/:id,556,50,0.09,0.83,0.71,1.00
18,/admin/invoices/create,812,160,0.20,0.83,0.77,0.91
10,/admin/customers/:id,685,86,0.13,0.83,0.74,0.94
7,/admin/orders,649,221,0.34,0.82,0.92,0.73
11,/admin/reports,804,254,0.32,0.81,0.80,0.82


Total routes: 20
Average tokens per route: 706.0
Average unique tokens per route: 169.2
Average unique ratio: 0.24
Average F1 score: 0.79
Average precision: 0.81
Average recall: 0.79
Top-5 accuracy: 0.97

=== Correlations between tokens and metrics ===
Correlation unique_ratio vs precision: 0.046
Correlation tokens vs recall: 0.134
Correlation unique_tokens vs f1-score: -0.555

=== Test query ===
Query: Create a new user in the system
Path: /admin/users/create, Score: -0.18
Context: admin users create users create erp admin new account add user Adding user account Need add new user specific roles New user setup create new user account Creaeting new user add user New user creation Setup new user New user registration New user account creation New user onboarding New user creation Create new employee profile New employee profile User creation form Adding new user account New user setup process New user creation process New employee setup Set new user account finance department Add user d

In [14]:
_models_path = "../build/models"
knn_engine_path = knn_engine.save_model("knn_model", _models_path)
knn_model_size_mb = os.path.getsize(knn_engine_path)
print(f"Model saved to {knn_engine_path}, size: {knn_model_size_mb} MB")

svc_engine_path = svc_engine.save_model("svc_model", _models_path)
svc_model_size_mb = os.path.getsize(svc_engine_path)
print(f"Model saved to {svc_engine_path}, size: {svc_model_size_mb} MB")

Model saved to ../build/models/knn_model.pkl, size: 451263377 MB
Model saved to ../build/models/svc_model.pkl, size: 451484365 MB
